# **Loading data**

In [ ]:
import pandas as pd
import ast

# Load the dataset
file_path = '/content/TRADES_CopyTr_90D_ROI.csv'
df = pd.read_csv(file_path)

# Step 1: Drop rows with missing 'Trade_History'
df_clean = df.dropna(subset=['Trade_History'])

# Step 2: Convert 'Trade_History' from string to list of dictionaries
df_clean['Trade_History'] = df_clean['Trade_History'].apply(ast.literal_eval)

# Step 3: Extract relevant fields from 'Trade_History'
# Define a function to extract relevant fields
def extract_trade_info(trades):
    trade_list = []
    for trade in trades:
        trade_list.append({
            'time': trade.get('time'),
            'symbol': trade.get('symbol'),
            'side': trade.get('side'),
            'price': trade.get('price'),
            'quantity': trade.get('quantity'),
            'realizedProfit': trade.get('realizedProfit')
        })
    return trade_list

# Apply the extraction function to each account's trade history
df_clean['Parsed_Trades'] = df_clean['Trade_History'].apply(extract_trade_info)

# Step 4: Expand the parsed trades into a new dataframe
expanded_trades = df_clean.explode('Parsed_Trades')
expanded_trades = expanded_trades.dropna(subset=['Parsed_Trades'])

# Convert the parsed dictionary back to separate columns
expanded_trades = pd.concat([expanded_trades.drop(['Parsed_Trades'], axis=1),
                             expanded_trades['Parsed_Trades'].apply(pd.Series)], axis=1)

# The data is now cleaned and ready for further analysis
# Display the cleaned dataframe
print(expanded_trades.head())


In [ ]:
import pandas as pd
import numpy as np

# Assuming `expanded_trades` is the cleaned dataframe from the previous steps

# Group by 'Port_IDs' to calculate the metrics for each account
grouped = expanded_trades.groupby('Port_IDs')

# Calculate metrics for each account
metrics = pd.DataFrame()

# Calculate PnL
metrics['Total_PnL'] = grouped['realizedProfit'].sum()

# Calculate ROI (assuming 'price' * 'quantity' gives total investment)
metrics['Total_Investment'] = grouped.apply(lambda x: (x['price'] * x['quantity']).sum())
metrics['ROI'] = (metrics['Total_PnL'] / metrics['Total_Investment']) * 100

# Calculate total number of positions
metrics['Total_Positions'] = grouped.size()

# Calculate win positions (where realizedProfit > 0)
metrics['Win_Positions'] = grouped.apply(lambda x: (x['realizedProfit'] > 0).sum())

# Calculate Win Rate
metrics['Win_Rate'] = (metrics['Win_Positions'] / metrics['Total_Positions']) * 100

# Calculate Sharpe Ratio (assuming the average return over all positions)
metrics['Mean_Return'] = grouped['realizedProfit'].mean()
metrics['Std_Return'] = grouped['realizedProfit'].std()
risk_free_rate = 0.02  # Can be adjusted
metrics['Sharpe_Ratio'] = (metrics['Mean_Return'] - risk_free_rate) / metrics['Std_Return']

# Calculate Maximum Drawdown (MDD)
def calculate_mdd(profits):
    cumulative_returns = profits.cumsum()
    peak = cumulative_returns.cummax()
    drawdown = peak - cumulative_returns
    max_drawdown = drawdown.max()
    return max_drawdown

metrics['MDD'] = grouped['realizedProfit'].apply(calculate_mdd)

# Rank accounts based on a combination of metrics (you can adjust the weights)
metrics['Score'] = (metrics['ROI'] * 0.3 +
                    metrics['Sharpe_Ratio'] * 0.3 +
                    metrics['Win_Rate'] * 0.2 -
                    metrics['MDD'] * 0.2)

# Sort by score to get top 20 accounts
top_20_accounts = metrics.sort_values(by='Score', ascending=False).head(20)

# Display the top 20 accounts
print(top_20_accounts)

# Save the metrics to a CSV
metrics.to_csv('calculated_metrics.csv')
top_20_accounts.to_csv('top_20_accounts.csv')


# **PLOTING**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot ROI for top 20 accounts
plt.figure(figsize=(10, 6))
sns.barplot(x=top_20_accounts.index, y='ROI', data=top_20_accounts, palette='viridis')
plt.xticks(rotation=45)
plt.title('Top 20 Accounts by ROI')
plt.xlabel('Port_IDs')
plt.ylabel('ROI (%)')
plt.tight_layout()
plt.show()

# Plot PnL for top 20 accounts
plt.figure(figsize=(10, 6))
sns.barplot(x=top_20_accounts.index, y='Total_PnL', data=top_20_accounts, palette='coolwarm')
plt.xticks(rotation=45)
plt.title('Top 20 Accounts by Profit and Loss (PnL)')
plt.xlabel('Port_IDs')
plt.ylabel('Total PnL ($)')
plt.tight_layout()
plt.show()

# Distribution of Win Rate
plt.figure(figsize=(8, 6))
sns.histplot(metrics['Win_Rate'], bins=20, kde=True, color='blue')
plt.title('Distribution of Win Rate')
plt.xlabel('Win Rate (%)')
plt.ylabel('Frequency')
plt.show()



# **correlation matrix**

In [ ]:
# Calculate correlation matrix
corr = metrics[['ROI', 'Total_PnL', 'Sharpe_Ratio', 'MDD', 'Win_Rate']].corr()

# Plot a heatmap to visualize correlations
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Between Metrics')
plt.show()


In [ ]:
# Plot cumulative PnL for a specific account
account_id = top_20_accounts.index[0]  # Example: top account
account_trades = expanded_trades[expanded_trades['Port_IDs'] == account_id]

# Calculate cumulative PnL
account_trades['Cumulative_PnL'] = account_trades['realizedProfit'].cumsum()

plt.figure(figsize=(10, 6))
plt.plot(account_trades['time'], account_trades['Cumulative_PnL'], marker='o', color='green')
plt.title(f'Cumulative PnL for Account {account_id}')
plt.xlabel('Time')
plt.ylabel('Cumulative PnL ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# **Evaluated the model**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Example: Classify accounts into winners (Win Rate > 50%) or not
metrics['Is_Winner'] = metrics['Win_Rate'] > 50

# Select features and target
X = metrics[['ROI', 'Total_PnL', 'Sharpe_Ratio', 'MDD']]
y = metrics['Is_Winner']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Replace infinite values with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
# Fill NaN values with the mean of the column
X_train.fillna(X_train.mean(), inplace=True)

# Train a Random Forest Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


In [ ]:
# Save final analysis to Excel
with pd.ExcelWriter('final_trade_analysis.xlsx') as writer:
    metrics.to_excel(writer, sheet_name='All Metrics')
    top_20_accounts.to_excel(writer, sheet_name='Top 20 Accounts')


In [ ]:
# Save metrics dataframe to a pickle file
metrics.to_pickle('metrics.pkl')

# Save top 20 accounts dataframe to a pickle file
top_20_accounts.to_pickle('top_20_accounts.pkl')

print("Data saved to pickle files successfully.")


In [ ]:
import pickle

# Save multiple objects into a pickle file
with open('analysis_data.pkl', 'wb') as f:
    pickle.dump({'metrics': metrics, 'top_20_accounts': top_20_accounts}, f)

print("Data saved to 'analysis_data.pkl'.")


In [ ]:
# Load the saved metrics dataframe
metrics = pd.read_pickle('metrics.pkl')

# Load top 20 accounts dataframe
top_20_accounts = pd.read_pickle('top_20_accounts.pkl')

# Or, if using pickle module with multiple objects
with open('analysis_data.pkl', 'rb') as f:
    data = pickle.load(f)
    metrics = data['metrics']
    top_20_accounts = data['top_20_accounts']

print("Data loaded from pickle files successfully.")
